# Apriori

## Importing the libraries

In [1]:
 !pip install apyori


  Created wheel for apyori: filename=apyori-1.1.2-py3-none-any.whl size=5975 sha256=2e050d3159563e35cf325fad1029d05735ecfc0ef3591450620b02682251099d
  Stored in directory: /root/.cache/pip/wheels/cb/f6/e1/57973c631d27efd1a2f375bd6a83b2a616c4021f24aab84080
Successfully built apyori


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

## Data Preprocessing

In [3]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
data=pd.read_csv('/kaggle/input/market-basket-optimisationcsv/Market_Basket_Optimisation.csv')

data.head(5)

/kaggle/input/market-basket-optimisationcsv/Market_Basket_Optimisation.csv


,shrimp,almonds,avocado,vegetables mix,green grapes,whole weat flour,yams,cottage cheese,energy drink,tomato juice,low fat yogurt,green tea,honey,salad,mineral water,salmon,antioxydant juice,frozen smoothie,spinach,olive oil
0,burgers,meatballs,eggs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,chutney,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,turkey,avocado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,mineral water,milk,energy bar,whole wheat rice,green tea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,low fat yogurt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
len(data)

7500

In [5]:
## We need a list of transactions
transactions=[]
for i in range(0,7500): ## rows
    transactions.append([str(data.values[i,j]) for j in range (0,20) ])# Columns 
## must be strings

In [6]:
#transactions

## Training the Apriori model on the dataset

In [7]:
from apyori import apriori

rules=apriori(transactions=transactions,
              min_support=0.003,## (min 3 times per day  * 7 days)/7501 
              min_confidence=0.2,## you can try different values depending on the business requirements
              min_lift = 3,  ## relevance of the rule... good lift is at least 3. Rules below 3 are not relevant--based on the experience
              min_length=2, ## 2 products to have at the end, A and B
              max_length=2) 

In [8]:
rules

<generator object apriori at 0x7f6a3cc0ff50>

## Visualising the results

### Displaying the first results coming directly from the output of the apriori function

In [9]:
results=list(rules) ## we put rules into a list
#results

In [10]:
results[5]

RelationRecord(items=frozenset({'ground beef', 'tomato sauce'}), support=0.005333333333333333, ordered_statistics=[OrderedStatistic(items_base=frozenset({'tomato sauce'}), items_add=frozenset({'ground beef'}), confidence=0.37735849056603776, lift=3.840147461662528)])

In [11]:
## 0.004% that french fries and almonds will appear in the basket
## o.21% that french fries will appear at the basket if almonds are also bought

### Putting the results well organised into a Pandas DataFrame

In [12]:
def inspect(results):
    lhs         = [tuple(result[2][0][0])[0] for result in results]
    rhs         = [tuple(result[2][0][1])[0] for result in results]
    supports    = [result[1] for result in results]
    confidences = [result[2][0][2] for result in results]
    lifts       = [result[2][0][3] for result in results]
    return list(zip(lhs, rhs, supports, confidences, lifts))


    
resultsinDataFrame = pd.DataFrame(inspect(results), columns = ['Left Hand Side', 'Right Hand Side', 'Support', 'Confidence', 'Lift'])

### Displaying the results non sorted

In [13]:
resultsinDataFrame

,Left Hand Side,Right Hand Side,Support,Confidence,Lift
0,light cream,chicken,0.004533,0.290598,4.843305
1,mushroom cream sauce,escalope,0.005733,0.300699,3.790327
2,pasta,escalope,0.005867,0.372881,4.700185
3,fromage blanc,honey,0.003333,0.245098,5.178128
4,herb & pepper,ground beef,0.016000,0.323450,3.291555
5,tomato sauce,ground beef,0.005333,0.377358,3.840147
6,light cream,olive oil,0.003200,0.205128,3.120612
7,whole wheat pasta,olive oil,0.008000,0.271493,4.130221
8,pasta,shrimp,0.005067,0.322034,4.514494


### Displaying the results sorted by descending lifts

In [14]:
resultsinDataFrame.nlargest(n=10, columns="Lift") ## 10 rows

,Left Hand Side,Right Hand Side,Support,Confidence,Lift
3,fromage blanc,honey,0.003333,0.245098,5.178128
0,light cream,chicken,0.004533,0.290598,4.843305
2,pasta,escalope,0.005867,0.372881,4.700185
8,pasta,shrimp,0.005067,0.322034,4.514494
7,whole wheat pasta,olive oil,0.008000,0.271493,4.130221
5,tomato sauce,ground beef,0.005333,0.377358,3.840147
1,mushroom cream sauce,escalope,0.005733,0.300699,3.790327
4,herb & pepper,ground beef,0.016000,0.323450,3.291555
6,light cream,olive oil,0.003200,0.205128,3.120612
